In [1]:
import pandas as pd
import numpy as np
import time
import arrow

In [2]:
job_start_time = time.time()
df_train = pd.read_csv('../../dataset/meal_service/train.csv')
df_raining = pd.read_csv('../../dataset/meal_service/raining_stats.csv')
df_temperature = pd.read_csv('../../dataset/meal_service/temperature_stats.csv')
df_test = pd.read_csv('../../dataset/meal_service/test.csv')
df_result_date = pd.DataFrame(df_test['일자'])

In [4]:
count, bins_dividers = np.histogram(df_temperature['avg_temperature'], bins=7)

bin_names = [1, 2, 3, 4, 5, 6, 7]
df_temperature['avg_temp_binned'] = pd.cut(x=df_temperature['avg_temperature'],
                                           bins=bins_dividers,
                                           labels=bin_names,
                                           include_lowest = True)
df_temperature = df_temperature.drop(['avg_temperature', 'min_temperature', 'max_temperature'], axis='columns')

count, bins_dividers = np.histogram(df_raining['precipitation'], bins=4)

bin_names = [1, 2, 3, 4]
df_raining['precipitation_binned'] = pd.cut(x=df_raining['precipitation'],
                                            bins=bins_dividers,
                                            labels=bin_names,
                                            include_lowest = True)

df_raining['precipitation_binned'] = df_raining['precipitation_binned'].cat.set_categories([0, 1, 2, 3, 4])
df_raining.loc[df_raining['precipitation'] == 0, 'precipitation_binned'] = 0
df_raining = df_raining.drop(['precipitation'], axis='columns')

In [5]:
df_train = pd.merge(left=df_train, right=df_raining, how='left', on='일자')
df_train = pd.merge(left=df_train, right=df_temperature, how='left', on='일자')
df_test = pd.merge(left=df_test, right=df_raining, how='left', on='일자')
df_test = pd.merge(left=df_test, right=df_temperature, how='left', on='일자')


In [6]:
def get_is_holiday_lists(df):
    # pre-processing    
    is_yesterday_holiday_list = []
    is_tomorrow_holiday_list = []

    for i in range (0, len(df)):
        if df.iloc[i]['요일'] == '월':
            is_yesterday_holiday_list.append(1)
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '화' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '화':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '수' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '월' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '수':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '목' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '화' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '목':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '금' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '수' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '금':
            is_tomorrow_holiday_list.append(1)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '목' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
                
    return is_yesterday_holiday_list, is_tomorrow_holiday_list

In [7]:
delicious_dish_list = ['무피클/사과주스', '꼬지어묵탕', '콘치즈구이', '야채볶음밥', '군고구마', '나물비빔밥', '비빔야채만두', '모듬튀김*칠리소스', '들깨미역국', '양념고추지', '무피클', '깍두기', '메추리알치즈떡볶이', '단무지', '양배추샐러드/귤', '잔치국수', '유부장국', '한방소갈비찜', '코코넛새우튀김', '해물까스*칠리소스', '볶음밥', '가지무침', '꼬치어묵매운탕', '푸실리샐러드', '목살필라프', '도라지오이초무침', '우동장국', '통새우김밥', '로제파스타', '류산슬', '파채*소스/쌈무', '수제피클/오렌지', '아삭고추무침', '오므라이스', '봄동겉절이', '시금치국', '가쯔오국', '찐만두', '양송이스프', '수제돈까스*칠리소스', '락교', '미역장국', '쫄면야채무침', '브로컬리크림스프', '요플레', '채소볶음우동', '오이피클/음료', '시금치샐러드', '채소튀김*장', '볶음밥*짜장소스', '물만두국', '충무김밥', '스프', '옛날돈가스', '건파래볶음', '된장찌개', '깐풍육', '부대찌개', '오징어초무침', '국물떡볶이', '어묵국', '닭갈비볶음밥', '구슬떡볶이', '돈육강정', '조각티라미수', '찐옥수수', '펜네베이컨샐러드', '설렁탕', '콩나물밥*양념장', '열무된장나물', '유부초밥', '케이준샐러드', '핫도그', '토마토스파게티', '단무지무침', '잡채밥/짜장소스', '배추겉절이', '야채튀김', '메추리알조림', '찹쌀순대볶음', '옛날돈까스', '부추만두', '만두튀김', '간장깻잎지/콘샐러드', '수제마늘바게트', '떡볶이', '자반고등어찜', '부추전', '새우볶음밥', '양장피잡채', '수제핫도그', '유산슬', '오복지무침', '연두부*장', '쇠고기육전', '수제돈까스', '타워함박스테이크', '꽃맛살샐러드', '장아찌/피크닉', '닭매운찜', '김주먹밥', '소고기퀘사디아', '야채스틱/과일', '오징어무침', '콘슬로우', '비타민흑임자샐러드', '시금치나물', '돈육고추장볶음', '떡갈비주먹밥/추가밥', '시금치흑임자샐러드', '과일', '맑은계란국', '궁중떡찜', '언양식불고기', '멕시칸샐러드', '가쯔오장국', '탕수육', '홍시드레싱샐러드/음료', '짜장밥', '미니햄버거', '바나나', '훈제오리냉채', '딤섬', '우동', '수박', '마늘쫑무침', '버섯영양밥*우렁양념장', '생과일플레인샐러드', '흑미밥', '단호박스프', '유기농식혜', '해물부추전', '쇠고기볶음밥', '샐러드*오리엔탈D', '미트볼파스타*추가밥', '치킨마요덮밥', '해물까스*와사비소스', '해물콩나물찜', '(New)마약계란장조림', '미역국', '또띠아피자', '크림스프', '김밥', '키위드레싱샐러드', '수제오이피클', '옥수수콘치즈', '양배추피클', '오이미역무침', '고기짬뽕국', '쇠고기퀘사디아', '양배추샐러드', '순대야채볶음', '얼갈이나물된장무침', '잡채', '크림스프/추가밥', '단무지/음료', '콩나물무침', '대패삼겹*파채', '로제스파게티', '수제고로케', '오이생채/음료', '팽이장국', '크림카레우동', '만두찜', '카레덮밥', '칼국수', '수제무말랭이무침', '해물굴소스볶음밥', '알리오올리오파스타', '오징어젓무침', '시리얼과일샐러드', '식혜,오렌지,바나나', '빠에야', '옛날왕돈가스', '작은밥', '양배추샐러드/모닝빵', '떡만두국', '아오리사과', '찐옥수수/무피클', '씨리얼과일샐러드', '미소장국', '콘샐러드', '단호박범벅', '다시마채무초무침', '새우튀김우동', '맑은국', '냉모밀국수', '콥샐러드', '오렌지', '샤워크림새우', '미니잔치국수', '생선까스*칠리소스', '동태전/새송이전', '수제칠리핫도그', '닭오븐구이', '어묵우동', '석박지', '초계국수', '김말이,고추튀김', '감자튀김*케찹', '쌀국수', '컵주스', '작은밥/맑은국', '웨지감자', '사천식탕수육', '파인애플볶음밥', '하와이안샐러드', '짬뽕', '꼬들단무지/음료', '수제피클', '꼬마김밥*겨자장', '시저샐러드', '브로콜리숙회*초장', '해쉬포테이토*케찹', '고구마치즈구이', '두부조림', '알감자버터구이', '음료', '과일주스', '계란국', '새우까스*와사비소스', '돈까스김밥', '마파두부덮밥', '미니우동']

In [8]:
def get_is_corona_list(df):
    corona_list = []
    corona_start = arrow.get('2020-03-01')
    for i in range(0, len(df)):
        date = arrow.get(df.iloc[i]['일자'])
        if date >= corona_start:
            corona_list.append(1)
        else:
            corona_list.append(0)
    
    return corona_list

In [9]:
def is_last_wed_of_month(date):
    curr = arrow.get(date)
    if curr.weekday() != 2:
        # not wednesday
        return False
    else:
        curr_month = curr.month
        if curr_month != curr.shift(days=7).month:
            # last wednesday of the month
            return True
        else:
            return False
        
def get_is_last_wed_list(df):
    is_last_wed_list = []
    for i in range(0, len(df)):
        date = df.iloc[i]['일자']
        if is_last_wed_of_month(date):
            is_last_wed_list.append(1)
        else:
            is_last_wed_list.append(0)
    
    return is_last_wed_list

In [10]:
def get_month_list(df):
    month_list = []
    for i in range(0, len(df)):
        month = arrow.get(df.iloc[i]['일자']).month
        month_list.append(month)
    
    return month_list

In [11]:
def get_day_list(df):
    day_list = []
    for i in range(0, len(df)):
        day = arrow.get(df.iloc[i]['일자']).day
        day_list.append(day)
    
    return day_list

In [12]:
def seperate_and_processing_menu_str(menu_row):
    # split by space
    splits = menu_row.split(' ')
    menu = []
    is_new_menu = False
    for dish in splits:
        if len(dish) > 1:
            # find (New)
            if '(New)' in dish:
                menu.append(dish.split('(New)')[0] + dish.split('(New)')[1])
                is_new_menu = True
            elif '(' not in dish and ')' not in dish:
                menu.append(dish)
    if len(menu) <= 3:
        # no menu today
        menu = ['-']
    
    return menu, is_new_menu

In [13]:
udf_delicious_lunch = [
    # meq than avg 1000, cnt 7
    '북어국',
    '돈육간장불고기',
    '모듬쌈',
    '모듬쌈*쌈장',
    '해파리겨자채',
    '옥수수계란찜',
    '쇠고기숙주볶음',
    '두부양념조림',
    '황태국',
    '유자청제육볶음',
    '소불고기',
    '홍어채무침',
    '골뱅이무침*소면',
    '제육볶음',
    '계란찜',
    '버섯불고기',
    '어묵국',
    '한방소갈비찜',
    '쫄면채소무침',
    '모둠쌈',
    '돈육김치볶음',
    '돈육간장불고기'
]

udf_delicious_dinner = [
    # meq than avg 540 cnt 7
    '수박',
    '락교',
    '우동',
    '로제스파게티',
    '오므라이스',
    '후난식볶음밥',
    '통도라지구이',
    '미니햄버거',
    '김치볶음밥',
    '가쯔오장국',
    '콘샐러드',
    '무채국',
    '해파리냉채',
    '콥샐러드',
    '취나물',
    '비트무생채'
]    

In [14]:
def get_delicious_lunch_menu(lunch_menu):
    is_delicious_lunch = []
    for menu in lunch_menu:
        is_found = False
        for dish in menu:
            if dish in udf_delicious_lunch:
                is_delicious_lunch.append(1)
                is_found = True
                break
        if not is_found:
            is_delicious_lunch.append(0)
    return is_delicious_lunch

def get_delicious_dinner_menu(dinner_menu):
    is_delicious_dinner = []
    for menu in dinner_menu:
        is_found = False
        for dish in menu:
            if dish in udf_delicious_dinner:
                is_delicious_dinner.append(1)
                is_found = True
                break
        if not is_found:
            is_delicious_dinner.append(0)
    return is_delicious_dinner

In [15]:
def get_menu_info(df):
    is_new_lunch_menu = []
    is_new_dinner_menu = []
    lunch_score = []
    dinner_score = []
    lunch_menus = []
    dinner_menus = []
    for i in range(0, len(df)):
        row_lunch = df.iloc[i]['중식메뉴']
        row_dinner = df.iloc[i]['석식메뉴']
        
        lunch_menu, new_lunch_menu = seperate_and_processing_menu_str(row_lunch)
        lunch_menus.append(lunch_menu)
        lunch_score.append(get_menu_score(lunch_menu))
        is_new_lunch_menu.append(1) if new_lunch_menu else is_new_lunch_menu.append(0)
        
        dinner_menu, new_dinner_menu = seperate_and_processing_menu_str(row_dinner)
        dinner_menus.append(dinner_menu)
        dinner_score.append(get_menu_score(dinner_menu))
        is_new_dinner_menu.append(1) if new_dinner_menu else is_new_dinner_menu.append(0)
        
        # to do 
        # menu classification
    is_delicious_menu_lunch_list = get_delicious_lunch_menu(lunch_menus)
    is_delicious_menu_dinner_list = get_delicious_dinner_menu(dinner_menus)
    
    return is_new_lunch_menu, is_new_dinner_menu, is_delicious_menu_lunch_list, is_delicious_menu_dinner_list, lunch_score, dinner_score

In [16]:
def get_menu_score(menu):
    score = 0
    for dish in menu:
        if dish in delicious_dish_list:
            score = score + 1
    return score

In [17]:
def get_is_no_dinner_list(df):
    is_no_dinner_list = []
    for i in range(0, len(df)):
        is_no_dinner_list.append(1) if len(df.iloc[i]['석식메뉴']) <= 20 else is_no_dinner_list.append(0)
    
    return is_no_dinner_list

In [18]:
def pre_processing(df):
    yesterday_holiday_list, tommorow_holiday_list = get_is_holiday_lists(df)
    is_corona_list = get_is_corona_list(df)
    month_list = get_month_list(df)
    day_list = get_day_list(df)
    is_no_dinner_list = get_is_no_dinner_list(df)
    is_new_lunch_list, is_new_dinner_list, is_delicious_menu_lunch_list, is_delicious_menu_dinner_list, lunch_score_list, dinner_score_list = get_menu_info(df)
    is_last_wed_list = get_is_last_wed_list(df)

    is_yesterday_holiday = pd.DataFrame({'is_yesterday_holiday': yesterday_holiday_list})
    is_tomorrow_holiday = pd.DataFrame({'is_tomorrow_holiday': tommorow_holiday_list})
    is_corona = pd.DataFrame({'is_corona': is_corona_list})
    month = pd.DataFrame({'month': month_list})
    day = pd.DataFrame({'day': day_list})
    no_dinner = pd.DataFrame({'no_dinner': is_no_dinner_list})
    is_new_lunch = pd.DataFrame({'is_new_lunch': is_new_lunch_list})
    is_new_dinner = pd.DataFrame({'is_new_dinner': is_new_dinner_list})
    is_delicious_lunch = pd.DataFrame({'is_delicious_lunch': is_delicious_menu_lunch_list})
    is_delicious_dinner = pd.DataFrame({'is_delicious_dinner': is_delicious_menu_dinner_list})
    lunch_score = pd.DataFrame({'lunch_score': lunch_score_list})
    dinner_score = pd.DataFrame({'dinner_score': dinner_score_list})
    
    #is_last_wed = pd.DataFrame({'is_last_wed': is_last_wed_list})

    df = df.join(is_yesterday_holiday)
    df = df.join(is_tomorrow_holiday)
    df = df.join(is_corona)
    df = df.join(month)
    df = df.join(day)
    df = df.join(no_dinner)
    #df = df.join(is_new_lunch)
    df = df.join(is_new_dinner)
    #df = df.join(is_delicious_lunch)
    df = df.join(is_delicious_dinner)
    df = df.join(lunch_score)
    df = df.join(dinner_score)

    df = df.drop(['일자', '조식메뉴', '중식메뉴', '석식메뉴'],axis='columns')

    # 원핫인코딩
    df = pd.get_dummies(df, columns=['요일'])
    
    return df

In [19]:
df_train = pre_processing(df_train)

In [20]:
df_train.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,precipitation_binned,avg_temp_binned,is_yesterday_holiday,...,no_dinner,is_new_dinner,is_delicious_dinner,lunch_score,dinner_score,요일_금,요일_목,요일_수,요일_월,요일_화
0,2601,50,150,238,0.0,1039.0,331.0,0,2,1,...,0,0,0,0,1,0,0,0,1,0
1,2601,50,173,319,0.0,867.0,560.0,0,2,0,...,0,0,0,2,5,0,0,0,0,1
2,2601,56,180,111,0.0,1017.0,573.0,0,2,0,...,0,0,0,3,0,0,0,1,0,0
3,2601,104,220,355,0.0,978.0,525.0,0,2,0,...,0,0,1,2,4,0,1,0,0,0
4,2601,278,181,34,0.0,925.0,330.0,0,2,0,...,0,0,0,0,1,1,0,0,0,0


In [21]:
from sklearn.model_selection import train_test_split
import random

x1_train, x1_test, y1_train, y1_test = train_test_split(df_train.loc[:, [col for col in df_train.columns if col != '중식계' and col != '석식계']], df_train['중식계'],
                                                        random_state=random.randrange(1, 100))
x2_train, x2_test, y2_train, y2_test = train_test_split(df_train.loc[:, [col for col in df_train.columns if col != '중식계' and col != '석식계']], df_train['석식계'],
                                                        random_state=random.randrange(1, 100))

In [22]:

# AutoML을 이용한 ML 구현
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')


# 필요한 알고리즘을 불러와야 합니다!
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
# Ridge와 Lasso의 hyperparmeter: alpha
# Logistic Regressio의 hyperparameter: penalty와 regularization strength인 C값
from sklearn.svm import SVR
# SVR의 hyperparameter: epsilon, regularization C, gamma, kernel = 'rbf', 'poly', 'sigmoid'
from sklearn.neural_network import MLPRegressor
# hidden_layer_sizes = (100,) , (10, 10, ) 정도만 activation = 'relu', 'logistic'까지만, alpha =0.0001, solver = 'lbfgs', 'adam'까지만

In [23]:
pipe = Pipeline([('preprocessing', None), ('regressor', LinearRegression())])
pre_list = [StandardScaler(), MinMaxScaler(), None]
hyperparam_grid = [
    {'regressor': [LinearRegression()], 'preprocessing': pre_list},
    {'regressor': [MLPRegressor()], 'preprocessing': pre_list,
     'regressor__hidden_layer_sizes': [(100,) , (10, 10, )],
     'regressor__activation': ['relu', 'logistic'],
     'regressor__solver': ['lbfgs', 'adam'],
     'regressor__alpha': [0.0001, 0.01, 1]},
    {'regressor': [XGBRegressor(objective='reg:squarederror')],
     'preprocessing': pre_list,
     'regressor__learning_rate': [0.0, 0.1, 0.09, 0.089, 0.08],
     'regressor__booster': ['gbtree', 'gblinear', 'dart']}
]
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

grid1 = GridSearchCV(pipe, hyperparam_grid, scoring='neg_mean_absolute_error',
                   refit=True, cv=kfold)
grid2 = GridSearchCV(pipe, hyperparam_grid, scoring='neg_mean_absolute_error',
                   refit=True, cv=kfold)

In [24]:
grid1.fit(x1_train, y1_train)
print(grid1.best_estimator_)
print(grid1.best_params_)
print(-grid1.best_score_)
print(-grid1.score(x1_test, y1_test))

Pipeline(steps=[('preprocessing', MinMaxScaler()),
                ('regressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.08,
                              max_delta_step=0, max_depth=6, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=12, num_parallel_tree=1,
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])
{'preprocessing': MinMaxScaler(), 'regressor': XGBRegressor(base_score=None, booster='gbtree

In [ ]:
grid2.fit(x2_train, y2_train)
print(grid2.best_estimator_)
print(grid2.best_params_)
print(-grid2.best_score_)
print(-grid2.score(x2_test, y2_test))

In [ ]:
from sklearn.metrics import mean_absolute_error
print("중식 최종 결과: ", mean_absolute_error(y1_test, grid1.best_estimator_.predict(x1_test)))
print("석식 최종 결과: ", mean_absolute_error(y2_test, grid2.best_estimator_.predict(x2_test)))

In [ ]:
df_test = pre_processing(df_test)

lunch_cnt_list = grid1.best_estimator_.predict(df_test)
dinner_cnt_list = grid2.best_estimator_.predict(df_test)

In [ ]:
df_result_lunch = pd.DataFrame({'중식계': lunch_cnt_list})
df_result_dinner = pd.DataFrame({'석식계': dinner_cnt_list})

In [ ]:
df_result = df_result_date.join(df_result_lunch).join(df_result_dinner)

In [ ]:
df_result.to_csv('../../../../dataset/meal_service/result.csv', index=None)

In [ ]:
df_result

In [ ]:
print('job running time %f sec' % (time.time() - job_start_time ))